#  Events being lost in Chrome when navigating to an external URL

**Status :** The issue reported in [T258513](https://phabricator.wikimedia.org/T258513) seems to have been resolved. The events are in fact getting logged in the database however it does not show up in the Network tab due to a potential bug in the Chrome browser.

In [1]:
import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
import numpy as np
import pandas as pd
from tabulate import tabulate
from wmfdata import charting, hive, mariadb
from wmfdata.charting import comma_fmt, pct_fmt
from wmfdata.utils import df_to_remarkup, pct_str

You are using wmfdata v1.0.1, but v1.0.4 is available.

To update, run `pip install --upgrade git+https://github.com/neilpquinn/wmfdata/wmfdata.git@release`.

To see the changes, refer to https://github.com/neilpquinn/wmfdata/blob/release/CHANGELOG.md


## Donate events
Donate link is an external URL. Since this is of importance in our analysis we will first check if this event has been logged correctly.

In [5]:
# Donate link events by logged in/out users by day

count_donate_anon= hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.name as name,
  wiki AS wiki,
  event.isanon AS anonymous_user,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 AND day>19
  AND event.name LIKE 'n-sitesupport'
GROUP BY date_format(dt, "yyyy-MM-dd"),event.name, wiki, event.isanon 
""")

On July 23, Web-QA fired more than 400 events to Donate link on testwiki in total from 3 browsers. Out of this 120 were anonymous (logged out) and about 100 were logged in.
We can see greater number of events in the schema (which is always better than lesser). 

In [6]:
count_donate_anon.pivot_table(index=['name','wiki','date'], 
                                    columns=['anonymous_user']).fillna(0)          

events       
anonymous_user                     False  True 
name          wiki     date                    
n-sitesupport testwiki 2020-07-20    2.0   20.0
                       2020-07-21   80.0    0.0
                       2020-07-23  141.0  123.0

### by Browser

In [8]:
# Donate link events by logged in/out users by day

count_donate_anon_browser= hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.name as name,
  wiki AS wiki,
  event.isanon AS anonymous_user,
  useragent.browser_family AS browser,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 AND day>19
  AND event.name LIKE 'n-sitesupport'
GROUP BY date_format(dt, "yyyy-MM-dd"),event.name, wiki, event.isanon , useragent.browser_family
""")

Similarly, there are 40 or greater events fired on July 23, anonymously and logged in, from each of the 3 browsers : Chrome, Firefox, Safari. 

In [14]:
count_donate_anon_browser.pivot_table(index=['name','wiki','date', 'browser'], 
                                    columns=['anonymous_user']).fillna(0)          

events      
anonymous_user                             False True 
name          wiki     date       browser             
n-sitesupport testwiki 2020-07-20 Chrome     0.0  20.0
                                  Firefox    2.0   0.0
                       2020-07-21 Chrome    20.0   0.0
                                  Edge      20.0   0.0
                                  Firefox   20.0   0.0
                                  Safari    20.0   0.0
                       2020-07-23 Chrome    61.0  43.0
                                  Firefox   40.0  40.0
                                  Safari    40.0  40.0

### by SkinVersion

In [15]:
# Donate link events by logged in/out users by skinversion

count_donate_skinVersion_anon_day= hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.skinversion,
  event.name as name,
  wiki AS wiki,
  event.isanon AS anonymous_user,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 AND day>19
  AND event.name LIKE 'n-sitesupport'
GROUP BY date_format(dt, "yyyy-MM-dd"),event.skinversion, event.name, wiki, event.isanon 
""")

Per Web-QA, they fired 
- 60 events, logged in, on Legacy vector  
- 60 events, logged in, on new vector  
- 60 events, logged out, on Legacy vector  
- 60 events, logged out, on new vector  

Which have been recorded in the schema as expected

In [16]:
count_donate_skinVersion_anon_day.pivot_table(index=
                                              ['name','wiki','date','skinversion'], 
                                              columns=['anonymous_user']).fillna(0)

events      
anonymous_user                                 False True 
name          wiki     date       skinversion             
n-sitesupport testwiki 2020-07-20 1              0.0  10.0
                                  2              2.0  10.0
                       2020-07-21 2             80.0   0.0
                       2020-07-23 1             59.0  63.0
                                  2             82.0  60.0

For testing, we also looked at firing another external link, the Issue tracker. 

In [24]:
# Issue-tracker link events by logged in/out users by Browser

count_issuetracker_anon_browser= hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.name as name,
  wiki AS wiki,
  event.isanon AS anonymous_user,
  useragent.browser_family AS browser,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 AND day>19
  AND event.name LIKE 'n-Issue-tracker'
GROUP BY date_format(dt, "yyyy-MM-dd"),event.name, wiki, event.isanon , useragent.browser_family
""")

Of the 220 events fired by Web-QA we see greater than 250 events in the Db.

In [26]:
count_issuetracker_anon_browser.pivot_table(index=['name','wiki','date', 'browser'], 
                                    columns=['anonymous_user']).fillna(0)          

events      
anonymous_user                               False True 
name            wiki     date       browser             
n-Issue-tracker testwiki 2020-07-20 Chrome     0.0   1.0
                         2020-07-21 Chrome    20.0   0.0
                                    Edge      20.0   0.0
                                    Firefox   20.0   0.0
                                    Safari    20.0   0.0
                         2020-07-23 Chrome    52.0  41.0
                                    Firefox   40.0  40.0
                                    Safari    40.0  40.0
                         2020-07-24 Chrome     0.0   1.0

We also wanted to see if other external links have been fired in the test wikis :
- frwiktionary
- euwiki
- ptwikiversity


In [31]:
# External link events by logged in/out users by skinversion

count_extlinks_skinVersion_anon_day= hive.run("""
SELECT
  date_format(dt, "yyyy-MM-dd") AS date,
  event.skinversion,
  event.name as name,
  wiki AS wiki,
  event.isanon AS anonymous_user,
  COUNT(*) AS events
FROM event.desktopwebuiactionstracking
WHERE
  year = 2020 
  AND month=7 AND day>19
  AND event.name IN ( 
  'n-sitesupport',
  'n-Issue-tracker',
  'n-shoplink',
  'n-blogtext',
  't-upload',
  't-wikibase'
  )
AND wiki <> 'testwiki' 
GROUP BY date_format(dt, "yyyy-MM-dd"),event.skinversion, event.name, wiki, event.isanon 
""")

In [32]:
count_extlinks_skinVersion_anon_day.pivot_table(index=
                                              ['name','wiki','date','skinversion'], 
                                              columns=['anonymous_user']).fillna(0)

events      
anonymous_user                            False True 
name       wiki   date       skinversion             
t-upload   euwiki 2020-07-22 2              0.0   1.0
t-wikibase euwiki 2020-07-22 1             60.0   0.0
                             2              0.0   1.0
                  2020-07-23 1              1.0   0.0
                  2020-07-24 1             57.0   0.0

As such, we did not expect to see a lot of activity to sidebar links or external links in these small wikis. This resonates in the data as well